# Battle of the Neighborhoods

## IBM Data Science Coursera Capstone

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Import Necessary Libraries for later activities

In [1]:
import json
import requests
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported')

Libraries imported


### Import table from Wikipedia to Pandas dataframe

In [49]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_zips = pd.read_html(wiki_url, na_values = ['Not assigned'])
df_toronto_zips = toronto_zips[0]

### Drop NaN values and reset the index

In [50]:
df_toronto_zips.dropna(thresh=2, inplace=True)
df_toronto_zips.reset_index(drop=True).head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Display shape of the dataframe

In [48]:
df_toronto_zips.shape

(103, 3)